In [ ]:
!pip install implicit

В данном датасете имеется файл rating_final.csv с данными по посетителю, ресторану и рейтингу, который он ему поставил. Таким образом, мы можем решить задачу рекомендации. Эти данные можно использовать для коллаборативной фильтрации. Я выбрал метод ALS из-за его скорости и эффективности работы с разреженными матрицами.

Имеются также различные данные, связанные с особенностями ресторанов и посетителей. Их можно использовать для content-based рекомендации.

В реальности, особенно в случае работы с большими данными, переобучать модель при каждом изменении матрицы взаимодействий является слишком затратной операцией, поэтому я решил использовать двухэтапную модель ALS + lightgbm.

Тут ALS выступает в роли модели, которая занимается выдачей заданного числа кандидатов на основе оценок пользователей и ресторанов с оценками.

lightgbm же уже ранжирует их, используя content-фичи из остальных файлов. Его можно переобучать чаще, чем ALS

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from scipy.sparse import csr_matrix
from implicit.als import AlternatingLeastSquares
from collections import defaultdict
import lightgbm as lgb
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.metrics import ndcg_score

In [ ]:
IS_EVALUATING = True # Флаг, необходимый для вывода метрик

Загружаю данные по явным оценкам и строю матрицу взаимодействий.

In [ ]:
def load_data_ALS():
    ratings = pd.read_csv("rating_final.csv")

    # Преобразование userID и placeID в индексы для ALS
    user_ids = ratings['userID'].unique()
    user_to_index = {user: idx for idx, user in enumerate(user_ids)}
    place_ids = ratings['placeID'].unique()
    place_to_index = {place: idx for idx, place in enumerate(place_ids)}

    # Создание матрицы взаимодействий
    ratings['user_index'] = ratings['userID'].map(user_to_index)
    ratings['place_index'] = ratings['placeID'].map(place_to_index)
    ratings_matrix = csr_matrix(
        (ratings['rating'], (ratings['user_index'], ratings['place_index'])),
        shape=(len(user_ids), len(place_ids))
    )

    return ratings_matrix, user_to_index, place_to_index, ratings


In [ ]:
ratings_matrix, user_to_index, place_to_index, ratings = load_data_ALS()

Обучаю модель ALS, которую можно сохранить для дальнейших взаимодействий

In [ ]:
model = AlternatingLeastSquares(factors=50, iterations=50)
model.fit(ratings_matrix)

  0%|          | 0/50 [00:00<?, ?it/s]

Определяю функцию генерации кандидатов

In [ ]:
def generate_candidates(model, ratings_matrix, user_to_index, place_to_index, ratings, is_evaluating : bool):
    # Генерация кандидатов
    candidates = []
    user_ids = list(user_to_index.keys())
    place_ids = list(place_to_index.keys())

    for user_id in user_ids:
        user_index = user_to_index[user_id]
        rated_places = ratings[ratings['userID'] == user_id]['placeID'].unique()
        all_places = set(place_ids)
        if not is_evaluating:
          all_places = list(all_places - set(rated_places))

        place_indices = [place_to_index[p] for p in all_places]
        user_vector = model.user_factors[user_index]
        place_vectors = model.item_factors[place_indices]
        scores = np.dot(place_vectors, user_vector)

        for place_id, score in zip(all_places, scores):
            candidates.append({
                'userID': user_id,
                'placeID': place_id,
                'predicted_rating': score
            })

    return pd.DataFrame(candidates)

Генерирую датасет с кандидатами (посетитель, ресторан, предсказанный скор от ALS)

In [ ]:
candidates_df = generate_candidates(model, ratings_matrix, user_to_index, place_to_index, ratings, IS_EVALUATING)

Определяю функцию для отбора выбранного количества наиболее релевантных кандидатов из сгенерированного датасета.

In [ ]:
def get_top_candidates(user_id, n_top, candidates_df):
    # Фильтрация кандидатов по пользователю
    user_candidates = candidates_df[candidates_df['userID'] == user_id]

    if user_candidates.empty:
        print(f"Для пользователя {user_id} нет кандидатов.")
        return pd.DataFrame()

    # Сортировка и выбор топ-N
    result_df = user_candidates.sort_values('predicted_rating', ascending=False).head(n_top)

    result_df = result_df.drop_duplicates(subset=['placeID'])
    return result_df

Теперь приступаю к работе с content-based частью модели.

Загружаю все датасеты, отбрасываю невалидные и мерджу их по общим ключам. В случае если, например, у посетителя несколько предпочтений по кухне, то создастся соответствующее количество записей с оценками, хотя на самом деле оценка могла быть лишь одна.

Я пробовал группировать значения подобных признаков в списки и кодировать их при помощи Multi-Hot, однако в таком случае метрики значительно ухудшались, поэтому было принято решение оставить данные в таком виде.

In [ ]:
def load_full_data():
    ratings = pd.read_csv("rating_final.csv")

    print(f"Всего оценок: {len(ratings)}")
    print(f"Уникальных пользователей: {ratings['userID'].nunique()}")

    geoplaces = pd.read_csv("geoplaces2.csv", encoding='latin1')
    userprofile = pd.read_csv("userprofile.csv", encoding='latin1')

    userprofile = userprofile[userprofile['userID'].notna()]

    dfs = {
        'ratings': ratings,
        'geoplaces': geoplaces,
        'userprofile': userprofile,
        'usercuisine': pd.read_csv("usercuisine.csv"),
        'userpayment': pd.read_csv("userpayment.csv"),
        'chefmozcuisine': pd.read_csv("chefmozcuisine.csv"),
        'chefmozparking': pd.read_csv("chefmozparking.csv"),
        'chefmozaccepts': pd.read_csv("chefmozaccepts.csv")
    }

    full_data = dfs['ratings'].merge(
        dfs['geoplaces'], on='placeID', validate='many_to_one'
    ).merge(
        dfs['userprofile'], on='userID', validate='many_to_one'
    ).merge(
        dfs['usercuisine'], on='userID', suffixes=('', '_user')
    ).merge(
        dfs['userpayment'], on='userID'
    ).merge(
        dfs['chefmozcuisine'], on='placeID'
    ).merge(
        dfs['chefmozparking'], on='placeID'
    ).merge(
        dfs['chefmozaccepts'], on='placeID'
    )

    full_data['Rpayment'] = full_data['Rpayment'].fillna('unknown')
    return full_data

На этапе FE удаляю ненужные фичи(адрес, зип-код, url, fax и т.д.), а также те, которые значительно снижали метрики или никак не них не влияли.

Также использую ordinal_encoder для признаков, которые явно можно сопоставить с числами и onehot_encoder для остальных.

In [ ]:
def feature_engineering(df):
    cols_to_drop = [
        'the_geom_meter_x', 'the_geom_meter_y', 'name', 'address',
        'city', 'state', 'country', 'fax', 'zip', 'url', 'placeID', 'Rcuisine_user', 'payment',
        'ratings', 'latitude', 'longitude', 'height', 'other_services'
    ]
    df = df.drop(columns=[c for c in cols_to_drop if c in df.columns], errors='ignore')

    # Ordinal Encoding
    ordinal_map = {
        'price': {'low': 1, 'medium': 2, 'high': 3, None: 2},
        'budget': {'low': 1, 'medium': 2, 'high': 3, None: 2},
        'alcohol': {'No_Alcohol_Served': 0, 'Wine-Beer': 1, 'Full_Bar': 2, None: 0},
        'smoking_area': {'none': 0, 'permitted': 1, 'section': 2, 'only': 3, None: 0},
        'dress_code': {'informal': 0, 'casual': 1, 'formal': 2, None: 1}
    }

    for col, mapping in ordinal_map.items():
        if col in df.columns:
            df[col] = df[col].map(mapping).fillna(0).astype('int8')

    # One-Hot Encoding
    categorical_cols = [
        'Rpayment', 'main_cuisine', 'parking_lot', 'franchise',
        'area', 'Rambience', 'accessibility', 'transport'
    ]
    categorical_cols = [c for c in categorical_cols if c in df.columns]
    df = pd.get_dummies(df, columns=categorical_cols, drop_first=True, dtype='int8')

    if 'userID' in df.columns and df['userID'].dtype not in ['int64', 'float64']:
        user_ids = df['userID']  # Запоминаю userID, т.к. его выкинет после select_dtypes

    # Фильтрация финальных признаков
    df = df.select_dtypes(include=['number'])
    df = df.loc[:, df.nunique() > 1]

    df['userID'] = user_ids # Возвращаю обратно

    return df

Определяю функцию для тренировки модели. Перед этим была протестирована XGBoost, но метрики на ней были хуже, поэтому я остановился на lightbm.
Оптимизацию гиперпараметров не проводил, подобрал вручную. Стоит отметить, что в гиперпараметрах в "objective" стоит параметр "lambdarank", который при обучении модели минимизирует указанную метрику. В данном случае "metric": "ndcg". То есть модель обучается ранжировать, а не угадывать рейтинги, как было бы в случае "regression" и "rmse".

In [ ]:
def train_lightgbm_model():
    full_data = load_full_data()
    processed_data = feature_engineering(full_data)

    y = processed_data['rating']
    X = processed_data.drop(columns=['rating', 'food_rating', 'service_rating', 'userID'], errors='ignore') # Выкидываю явные признаки и userID

    # Формирую group - количество элементов для каждого пользователя, нужны для lambdarank
    user_counts = processed_data['userID'].value_counts().sort_index()
    group = user_counts.tolist()

    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.15, random_state=42)

    train_users = processed_data.loc[X_train.index, 'userID']
    train_group = train_users.value_counts().sort_index().tolist()

    val_users = processed_data.loc[X_val.index, 'userID']
    val_group = val_users.value_counts().sort_index().tolist()

    params = {
        "objective": "lambdarank",
        "metric": "ndcg",
        "num_leaves": 20,
        "learning_rate": 0.01,
        "min_data_in_leaf": 30,
        "feature_fraction": 0.9,
        "verbosity": -1
    }


    train_dataset = lgb.Dataset(X_train, label=y_train, group=train_group) # train с группами
    val_dataset = lgb.Dataset(X_val, label=y_val, group=val_group, reference=train_dataset) # Валидационный с группами для train-а

    model = lgb.train(
        params,
        train_dataset,
        num_boost_round=1500,
        valid_sets=[val_dataset],
        callbacks=[lgb.early_stopping(50)]
    )

    return model, X.columns.tolist(), X_val, y_val

Создаю функцию, чтобы достать нужные данные по полученным рекомендациям ALS (кандидаты). Также сразу обрабатываю NaN-ы и нормирую числовые признаки

In [ ]:
def fill_candidates(candidates):
    user_features = pd.read_csv("userprofile.csv", encoding='latin1')
    place_features = pd.read_csv("geoplaces2.csv", encoding='latin1')

    filled = candidates.merge(
        user_features, on='userID', how='left', validate='many_to_one'
    ).merge(
        place_features, on='placeID', how='left', validate='many_to_one'
    )

    # NaN-ы
    for col in ['price', 'budget']:
        if col in filled.columns:
            filled[col] = filled[col].fillna('medium')

    if 'Rcuisine' in filled.columns:
        filled['main_cuisine'] = filled['Rcuisine'].fillna('other')
        filled = filled.drop(columns='Rcuisine')

    numeric_cols = ['birth_year', 'weight']
    for col in numeric_cols:
        if col in filled.columns:
            filled[col] = filled[col].fillna(filled[col].median())
            filled[col] = (filled[col] - filled[col].mean()) / filled[col].std()

    return filled

In [ ]:
def check_data_integrity():
    geoplaces = pd.read_csv("geoplaces2.csv", encoding='latin1')
    duplicates = geoplaces.duplicated('placeID').sum()
    if duplicates > 0:
        print(f"Найдено {duplicates} дубликатов placeID. Исправление...")
        geoplaces = geoplaces.drop_duplicates('placeID', keep='first')
        geoplaces.to_csv("geoplaces2.csv", index=False)
    return geoplaces['placeID'].unique()

Определяю функции метрик. Сначала также определил NDCG, но скоры lambdarank это не рейтинги, а потому я не уверен, что их можно использовать для подсчета dcg, но даже так значение метрики было довольно высоко (ndcg@10 = 0.85 при 10 кандидатах, ndcg@10 = 0.6 при 30 кандидатах)

In [ ]:
def precision_at_k(recommended, true_relevant, k):
    top_k = recommended[:k]
    relevant_in_top_k = len(set(top_k) & set(true_relevant))
    return relevant_in_top_k / k if k > 0 else 0

def recall_at_k(recommended, true_relevant, k):
    top_k = recommended[:k]
    relevant_in_top_k = len(set(top_k) & set(true_relevant))
    return relevant_in_top_k / len(true_relevant)

def average_precision_at_k(recommended, true_relevant, k):
    precisions = []
    relevant_count = 0

    for i, item in enumerate(recommended[:k], 1):
        if item in true_relevant:
            relevant_count += 1
            precisions.append(relevant_count / i)

    return sum(precisions) / len(true_relevant) if precisions else 0.0

In [ ]:
def print_metrics(avg_metrics):
    print(f"Оценено пользователей: {avg_metrics['users_processed']}")
    print(f"Precision@10: {avg_metrics['precision']:.4f}")
    print(f"Recall@10:    {avg_metrics['recall']:.4f}")
    print(f"MAP@10:       {avg_metrics['map']:.4f}")

Определяю функцию для подсчета метрик, где в ней же генерирую рекомендации для всех пользователей. Далее усредняю значения метрик по всем посетителям.

In [ ]:
def calculate_metrics_for_all_users(model, user_ids, k=10):
    ratings = pd.read_csv("rating_final.csv")
    place_info = pd.read_csv("geoplaces2.csv", encoding='latin1')

    metrics = {
        'precision': [],
        'recall': [],
        'map': [],
        'users_processed': 0
    }
    user_metrics = {}

    for user_id in user_ids:
        # Получаю истинные оценки пользователя
        user_ratings = (
            ratings[ratings['userID'] == user_id]
            .merge(place_info, on='placeID', how='left')
            .dropna(subset=['rating'])
            .sort_values('rating', ascending=False)
        )

        true_relevant = user_ratings['placeID'].tolist()
        true_scores = user_ratings['rating'].values.astype(float)

        # Генерация рекомендаций
        n_top = 10
        candidates = get_top_candidates(user_id, n_top, candidates_df)
        processed = feature_engineering(fill_candidates(candidates))
        processed = processed.reindex(columns=features, fill_value=0)

        scores = model.predict(processed)
        candidates['score'] = scores
        recommended = candidates.nlargest(k, 'score')['placeID'].tolist()

        # Расчёт метрик
        precision = precision_at_k(recommended, true_relevant, k)
        recall = recall_at_k(recommended, true_relevant, k)
        ap = average_precision_at_k(recommended, true_relevant, k)

        # Сохранение результатов
        user_metrics[user_id] = {
            'precision': precision,
            'recall': recall,
            'map': ap,
            'true_items': true_relevant[:5],
            'recommended': recommended[:k]
        }

        metrics['precision'].append(precision)
        metrics['recall'].append(recall)
        metrics['map'].append(ap)
        metrics['users_processed'] += 1


    # Усреднение метрик
    avg_metrics = {k: np.mean(v) if k != 'users_processed' else v
                  for k, v in metrics.items()}

    print_metrics(avg_metrics)

    return avg_metrics, user_metrics

Обучаю модель lightgbm на всех доступных данных.

In [ ]:
# Обучение модели lightgbm
model, features, _, _ = train_lightgbm_model()

Всего оценок: 1161
Уникальных пользователей: 138
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[1]	valid_0's ndcg@1: 0.982301	valid_0's ndcg@2: 0.985474	valid_0's ndcg@3: 0.985942	valid_0's ndcg@4: 0.986554	valid_0's ndcg@5: 0.986278


Считаю и вывожу метрики

In [ ]:
ratings = pd.read_csv("rating_final.csv")

user_ids_to_evaluate = ratings['userID'].unique()[:138] # Оцениваю по всем пользователям

avg_metrics, user_metrics = calculate_metrics_for_all_users(
    model, user_ids_to_evaluate, k=10)

<ipython-input-524-c24277f94eda>:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['userID'] = user_ids # Возвращаю обратно


Оценено пользователей: 138
Precision@10: 0.7022
Recall@10:    0.8568
MAP@10:       0.8347


Модель получает хорошие метрики от 10 до 30 кандидатов от ALS. Если бы данных было больше, то количество кандидатов можно было поднять.